# Libraries importieren

In [ ]:
import json
import scipy as sp
import pandas as pd
import random
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
from networkx.algorithms.community import girvan_newman
from networkx.algorithms.community import label_propagation_communities
from networkx.algorithms.community import louvain_communities

Ideen:
Centrality Analysis:
- Degree Centrality: Identify who interacts the most; they are socially active and likely to have allies.
- Betweenness Centrality: Find characters who act as bridges; eliminating them disrupts communication.
- Closeness Centrality: Spot the actors who can quickly spread influence throughout the network.
Cluster Analysis: Identify the communities or factions. The "enemy" may lead or significantly influence one cluster.
Netzwerk vergleichen von bös zu schlecht
- Link prediction über Episoden
- Analyse über Episoden

# Daten laden

In [ ]:
def get_data(path):
    file_path = path
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

In [ ]:
interaction_data = get_data("data/starwars-full-interactions-allCharacters.json")
mention_data = get_data("data/starwars-full-mentions.json")

# Daten analysieren

In [ ]:
# Anzahl der Knoten in den Interaktionsdaten
print("Anzahl der Knoten in den Interaktionsdaten: ", len(interaction_data['nodes']))
# Anzahl der Kanten in den Interaktionsdaten
print("Anzahl der Kanten in den Interaktionsdaten: ", len(interaction_data['links']))


In [ ]:
# Anzahl der Knoten in den Erwähnungsdaten
print("Anzahl der Knoten in den Erwähnungsdaten: ", len(mention_data['nodes']))
# Anzahl der Kanten in den Erwähnungsdaten
print("Anzahl der Kanten in den Erwähnungsdaten: ", len(mention_data['links']))


In [ ]:
# Knoten in den Interaktionsdaten
interaction_data['nodes']

In [ ]:
G = nx.Graph()

for node in interaction_data['nodes']:
    G.add_node(node['name'], value=node['value'], color=node['colour'])

for link in interaction_data['links']:
    source = interaction_data['nodes'][link['source']]['name']
    target = interaction_data['nodes'][link['target']]['name']
    G.add_edge(source, target, weight=link['value'])

pos = nx.spring_layout(G, seed=42, k=2)
node_colors = [G.nodes[node]['color'] for node in G.nodes]
node_sizes = [G.nodes[node]['value'] * 10 for node in G.nodes] #Skalierung mal 10

plt.figure(figsize=(12, 8))
nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=node_sizes)
nx.draw_networkx_edges(G, pos, width=1, alpha=0.5)
nx.draw_networkx_labels(G, pos, font_size=8, font_color='black', font_weight='bold')
plt.title("Charakter Interaktionen Netzwerk in Starwars", fontsize=18)
plt.axis('off')
plt.show()


Suchen ob es mehere Subgraphen gibt die untereinander nicht verbunden sind.

In [ ]:
connected_components = list(nx.connected_components(G))
num_components = len(connected_components)
component_sizes = [len(component) for component in connected_components]

plt.figure(figsize=(6, 5))
plt.bar(range(1, num_components + 1), component_sizes)
plt.xlabel("Graphen")
plt.ylabel("Anzahl der Knoten")
plt.title("Anzahl der Knoten in den verbundenen Komponenten")
plt.xticks(range(1, num_components + 1))
plt.show()

# Zweites Netzwerk untersuchen

In [ ]:
#print connected components 2
num_components, component_sizes

Eine Node scheint keine Interaktionen zu haben. Dies könnte ein Fehler in den Daten sein.

In [ ]:
connected_components[1]

Gold five ist ein Pilot der Rebellenallianz und hat keine Interaktionen. Dies könnte ein Fehler in den Daten sein.

In [ ]:
G.remove_node('GOLD FIVE')

# Analyse der Cliques

In [ ]:
cliques = list(nx.find_cliques(G))
clique_sizes = [len(clique) for clique in cliques]
max_clique_size = max(clique_sizes)
num_cliques = len(cliques)

plt.figure(figsize=(5, 4))
plt.hist(clique_sizes, bins=range(1, max_clique_size + 2), edgecolor='black', align='left')
plt.xlabel("Cliquen grösse")
plt.ylabel("Häufigkeit")
plt.title("Verteilung der Cliquengrösse")
plt.xticks(range(1, max_clique_size + 1))
plt.show()

In [ ]:
largest_cliques = [clique for clique in cliques if len(clique) == max_clique_size]
largest_clique = largest_cliques[0]
subgraph_clique = G.subgraph(largest_clique)
pos_clique = nx.spring_layout(subgraph_clique, seed=42)
plt.figure(figsize=(8, 8))
nx.draw_networkx_nodes(subgraph_clique, pos_clique, node_color='red', node_size=300)
nx.draw_networkx_edges(subgraph_clique, pos_clique, width=2)
nx.draw_networkx_labels(subgraph_clique, pos_clique, font_size=10, font_color='black')
plt.title(f"Grösste Cliquen (Grösse: {max_clique_size}) im Graphen", fontsize=16)
plt.axis('off')
plt.show()

# Interaction Datensatz

## Analyse Edge-Betweenness Clustering

In [ ]:
communities_generator = girvan_newman(G)
first_level_communities = next(communities_generator)
clusters = [list(community) for community in first_level_communities]

num_clusters = len(clusters)
cluster_sizes = [len(cluster) for cluster in clusters]

plt.figure(figsize=(6, 4))
plt.bar(range(1, num_clusters + 1), cluster_sizes, color='skyblue', edgecolor='black')
plt.xlabel("Cluster")
plt.ylabel("Anzahl der Knoten")
plt.title("Anzahl der Knoten in den Clustern")
plt.xticks(range(1, num_clusters + 1))
plt.show()

In [ ]:
cmap = matplotlib.colormaps.get_cmap("tab20")
colors = [cmap(i) for i in range(num_clusters)]
color_map = {}

for cluster_index, cluster_nodes in enumerate(clusters):
    for node in cluster_nodes:
        color_map[node] = colors[cluster_index]

node_colors = [color_map[node] for node in G.nodes]

plt.figure(figsize=(8, 8))
pos = nx.spring_layout(G, seed=42,k=0.8)
nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=100, alpha=0.8)
nx.draw_networkx_edges(G, pos, alpha=0.5)
nx.draw_networkx_labels(G, pos, font_size=8, font_color='black')
plt.title("Edge-Betweenness Clustering", fontsize=18)
plt.axis('off')
plt.show()

num_clusters, cluster_sizes

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 8))
axes = axes.flatten()

for i, cluster in enumerate(clusters):
    if i >= 1 * 3:
        break

    subgraph = G.subgraph(cluster)
    pos_subgraph = nx.spring_layout(subgraph, seed=42, k=0.8)
    ax = axes[i]
    nx.draw_networkx_nodes(subgraph, pos_subgraph, node_color=[colors[i]] * len(subgraph),
                           node_size=100, alpha=0.8, ax=ax)
    nx.draw_networkx_edges(subgraph, pos_subgraph, alpha=0.5, ax=ax)
    nx.draw_networkx_labels(subgraph, pos_subgraph, font_size=8, font_color='black', ax=ax)
    ax.set_title(f"Cluster {i + 1}", fontsize=12)
    ax.axis('off')

        # Hide any unused subplots
for j in range(i + 1, len(axes)):
    axes[j].axis('off')


## Label Propagation Community Detection

In [ ]:
label_propagation_communities_result = list(nx.algorithms.community.label_propagation_communities(G))
num_label_propagation_communities = len(label_propagation_communities_result)
distinct_colors_lp = [plt.cm.tab20(i / num_label_propagation_communities) for i in range(num_label_propagation_communities)]

label_propagation_color_map = {}
for i, community in enumerate(label_propagation_communities_result):
    for node in community:
        label_propagation_color_map[node] = distinct_colors_lp[i]

node_colors_label_propagation = [label_propagation_color_map[node] for node in G.nodes]

plt.figure(figsize=(15, 15))
pos_label_propagation = nx.spring_layout(G, seed=42, k=3)
nx.draw_networkx_nodes(G, pos_label_propagation, node_color=node_colors_label_propagation, node_size=100, alpha=0.9)
nx.draw_networkx_edges(G, pos_label_propagation, alpha=0.5)
nx.draw_networkx_labels(G, pos_label_propagation, font_size=8, font_color='black')
plt.title("Clusters nach Labelpropagation", fontsize=18)
plt.axis('off')
plt.show()


In [ ]:
# print the communities
len(label_propagation_communities_result)

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(16, 10))
axes = axes.flatten()

for i, community in enumerate(label_propagation_communities_result):
    if i >= 2 * 4:
        break

    subgraph = G.subgraph(community)
    pos_subgraph = nx.spring_layout(subgraph, seed=42, k=3)
    ax = axes[i]
    nx.draw_networkx_nodes(subgraph, pos_subgraph, node_color=[distinct_colors_lp[i]] * len(subgraph),
                           node_size=100, alpha=0.9, ax=ax)
    nx.draw_networkx_edges(subgraph, pos_subgraph, alpha=0.5, ax=ax)
    nx.draw_networkx_labels(subgraph, pos_subgraph, font_size=8, font_color='black', ax=ax)
    ax.set_title(f"Community {i + 1}", fontsize=12)
    ax.axis('off')

    # Hide any unused subplots
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

## Louvain Community Detection

In [ ]:
louvain_communities_result = louvain_communities(G, seed=42)
num_louvain_communities = len(louvain_communities_result)

louvain_colors = plt.cm.rainbow(range(num_louvain_communities))
louvain_color_map = {}

for community_index, community_nodes in enumerate(louvain_communities_result):
    for node in community_nodes:
        louvain_color_map[node] = louvain_colors[community_index % len(louvain_colors)]

node_colors_louvain = [louvain_color_map[node] for node in G.nodes]

distinct_colors = [plt.cm.tab20(i / num_louvain_communities) for i in range(num_louvain_communities)]
node_colors_louvain_distinct = []
for node in G.nodes:
    for i, community in enumerate(louvain_communities_result):
        if node in community:
            node_colors_louvain_distinct.append(distinct_colors[i])

plt.figure(figsize=(15, 15))
pos_louvain = nx.spring_layout(G, seed=42, k=3)
nx.draw_networkx_nodes(G, pos_louvain, node_color=node_colors_louvain_distinct, node_size=100, alpha=0.9)
nx.draw_networkx_edges(G, pos_louvain, alpha=0.5)
nx.draw_networkx_labels(G, pos_louvain, font_size=8, font_color='black')
plt.title("Louvain Community Detection", fontsize=18)
plt.axis('off')
plt.show()

In [ ]:
rows, cols = 2, 3
fig, axes = plt.subplots(rows, cols, figsize=(16, 10))
axes = axes.flatten()
for i, community in enumerate(louvain_communities_result):
    if i >= rows * cols:
        break
    subgraph = G.subgraph(community)
    pos_subgraph = nx.spring_layout(subgraph, seed=42, k= 3)
    ax = axes[i]
    nx.draw_networkx_nodes(subgraph, pos_subgraph, node_color=[distinct_colors[i]] * len(subgraph),
                           node_size=100, alpha=0.9, ax=ax)
    nx.draw_networkx_edges(subgraph, pos_subgraph, alpha=0.5, ax=ax)
    nx.draw_networkx_labels(subgraph, pos_subgraph, font_size=8, font_color='black', ax=ax)
    ax.set_title(f"Community {i + 1}", fontsize=12)
    ax.axis('off')

for j in range(i + 1, len(axes)):
    axes[j].axis('off')



In [ ]:
# Assuming `louvain_communities_result` is the list of communities
community_map = {}
for i, community in enumerate(louvain_communities_result):
    for node in community:
        community_map[node] = i

# Create a list to store inter-community edges
inter_community_edges = []

for u, v in G.edges():
    if community_map[u] != community_map[v]:  # Check if nodes belong to different communities
        inter_community_edges.append((u, v))

# Create a subgraph for inter-community edges
inter_community_subgraph = G.edge_subgraph(inter_community_edges)

# Visualize the graph with inter-community edges highlighted
plt.figure(figsize=(15, 15))
pos = nx.spring_layout(G, seed=42, k=3)
nx.draw_networkx_nodes(G, pos, node_color=node_colors_louvain_distinct, node_size=100, alpha=0.9)
nx.draw_networkx_edges(G, pos, alpha=0.2)  # Default edges
nx.draw_networkx_edges(inter_community_subgraph, pos, edge_color='red', alpha=0.8, width=2)  # Highlight inter-community edges
nx.draw_networkx_labels(G, pos, font_size=8, font_color='black')
plt.title("Louvain Communities and Inter-Community Connections", fontsize=18)
plt.axis('off')
plt.show()

# Mention Datasatz

In [ ]:
Gr = nx.Graph()

for node in mention_data['nodes']:
    Gr.add_node(node['name'], value=node['value'], color=node['colour'])

for link in mention_data['links']:
    source = mention_data['nodes'][link['source']]['name']
    target = mention_data['nodes'][link['target']]['name']
    Gr.add_edge(source, target, weight=link['value'])

pos = nx.spring_layout(Gr, seed=42, k=2)
node_colors = [Gr.nodes[node]['color'] for node in Gr.nodes]
node_sizes = [Gr.nodes[node]['value'] * 10 for node in Gr.nodes] #Skalierung mal 10

plt.figure(figsize=(12, 8))
nx.draw_networkx_nodes(Gr, pos, node_color=node_colors, node_size=node_sizes)
nx.draw_networkx_edges(Gr, pos, width=1, alpha=0.5)
nx.draw_networkx_labels(Gr, pos, font_size=8, font_color='black', font_weight='bold')
plt.title("Charakter Interaktionen Netzwerk in Starwars", fontsize=18)
plt.axis('off')
plt.show()


In [ ]:
connected_components = list(nx.connected_components(Gr))
num_components = len(connected_components)
component_sizes = [len(component) for component in connected_components]

plt.figure(figsize=(6, 5))
plt.bar(range(1, num_components + 1), component_sizes)
plt.xlabel("Graphen")
plt.ylabel("Anzahl der Knoten")
plt.title("Anzahl der Knoten in den verbundenen Komponenten")
plt.xticks(range(1, num_components + 1))
plt.show()

In [ ]:
label_propagation_communities_result = list(nx.algorithms.community.label_propagation_communities(Gr))
num_label_propagation_communities = len(label_propagation_communities_result)
distinct_colors_lp = [plt.cm.tab20(i / num_label_propagation_communities) for i in range(num_label_propagation_communities)]

label_propagation_color_map = {}
for i, community in enumerate(label_propagation_communities_result):
    for node in community:
        label_propagation_color_map[node] = distinct_colors_lp[i]

node_colors_label_propagation = [label_propagation_color_map[node] for node in Gr.nodes]

plt.figure(figsize=(15, 15))
pos_label_propagation = nx.spring_layout(Gr, seed=42, k=3)
nx.draw_networkx_nodes(Gr, pos_label_propagation, node_color=node_colors_label_propagation, node_size=100, alpha=0.9)
nx.draw_networkx_edges(Gr, pos_label_propagation, alpha=0.5)
nx.draw_networkx_labels(Gr, pos_label_propagation, font_size=8, font_color='black')
plt.title("Clusters nach Labelpropagation", fontsize=18)
plt.axis('off')
plt.show()


## Labelprop auf Mention

In [ ]:
# print the communities
len(label_propagation_communities_result)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

for i, community in enumerate(label_propagation_communities_result):
    if i >= 2 * 2:
        break

    subgraph = Gr.subgraph(community)
    pos_subgraph = nx.spring_layout(subgraph, seed=42, k=3)
    ax = axes[i]
    nx.draw_networkx_nodes(subgraph, pos_subgraph, node_color=[distinct_colors_lp[i]] * len(subgraph),
                           node_size=100, alpha=0.9, ax=ax)
    nx.draw_networkx_edges(subgraph, pos_subgraph, alpha=0.5, ax=ax)
    nx.draw_networkx_labels(subgraph, pos_subgraph, font_size=8, font_color='black', ax=ax)
    ax.set_title(f"Community {i + 1}", fontsize=12)
    ax.axis('off')

## Louvain

In [ ]:
louvain_communities_result = louvain_communities(Gr, seed=42)
num_louvain_communities = len(louvain_communities_result)

louvain_colors = plt.cm.rainbow(range(num_louvain_communities))
louvain_color_map = {}

for community_index, community_nodes in enumerate(louvain_communities_result):
    for node in community_nodes:
        louvain_color_map[node] = louvain_colors[community_index % len(louvain_colors)]

node_colors_louvain = [louvain_color_map[node] for node in Gr.nodes]

distinct_colors = [plt.cm.tab20(i / num_louvain_communities) for i in range(num_louvain_communities)]
node_colors_louvain_distinct = []
for node in Gr.nodes:
    for i, community in enumerate(louvain_communities_result):
        if node in community:
            node_colors_louvain_distinct.append(distinct_colors[i])

plt.figure(figsize=(15, 15))
pos_louvain = nx.spring_layout(Gr, seed=42, k=3)
nx.draw_networkx_nodes(Gr, pos_louvain, node_color=node_colors_louvain_distinct, node_size=100, alpha=0.9)
nx.draw_networkx_edges(Gr, pos_louvain, alpha=0.5)
nx.draw_networkx_labels(Gr, pos_louvain, font_size=8, font_color='black')
plt.title("Louvain Community Detection", fontsize=18)
plt.axis('off')
plt.show()

In [ ]:
rows, cols = 2, 3
fig, axes = plt.subplots(rows, cols, figsize=(16, 10))
axes = axes.flatten()
for i, community in enumerate(louvain_communities_result):
    if i >= rows * cols:
        break
    subgraph = Gr.subgraph(community)
    pos_subgraph = nx.spring_layout(subgraph, seed=42, k= 3)
    ax = axes[i]
    nx.draw_networkx_nodes(subgraph, pos_subgraph, node_color=[distinct_colors[i]] * len(subgraph),
                           node_size=100, alpha=0.9, ax=ax)
    nx.draw_networkx_edges(subgraph, pos_subgraph, alpha=0.5, ax=ax)
    nx.draw_networkx_labels(subgraph, pos_subgraph, font_size=8, font_color='black', ax=ax)
    ax.set_title(f"Community {i + 1}", fontsize=12)
    ax.axis('off')

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

